# Text Classification with Hugging Face 🎯

Text classification is one of the most common NLP tasks - assigning categories or labels to text documents. From spam detection to sentiment analysis, topic classification to intent recognition, this fundamental skill powers countless applications.

## What is Text Classification?

**Text Classification** assigns predefined categories to text:
- **Input**: Text document
- **Output**: Category label(s) with confidence scores
- **Examples**: Email spam detection, news categorization, customer feedback analysis

## Learning Objectives

By the end of this notebook, you'll know how to:
1. Use pre-trained classification models effectively
2. Fine-tune models on custom datasets
3. Implement multi-class and multi-label classification
4. Evaluate model performance with proper metrics
5. Handle imbalanced datasets and edge cases
6. Deploy classification models in production

Let's build some powerful classifiers! 🚀

In [ ]:
# Import essential libraries
import torch
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
from datasets import Dataset, load_dataset
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

# Check device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

print("Libraries loaded successfully!")

## 1. Pre-trained Classification Models

Let's explore various pre-trained models for different classification tasks:

In [ ]:
# Test different pre-trained classification models
print("🎯 Pre-trained Classification Models")
print("=" * 35)

# Load sentiment analysis pipeline
sentiment_classifier = pipeline("sentiment-analysis")

# Test texts
test_texts = [
    "I absolutely love this new product! It's amazing!",
    "This movie was terrible and boring. Waste of time.",
    "The weather is okay today, nothing special.",
    "I'm so excited about this opportunity!",
    "This is frustrating and makes me angry."
]

print("Sentiment Analysis Results:")
for i, text in enumerate(test_texts, 1):
    result = sentiment_classifier(text)[0]
    emoji = "😊" if result['label'] == 'POSITIVE' else "😞"
    print(f"{i}. {emoji} '{text}'")
    print(f"   → {result['label']} (confidence: {result['score']:.3f})")
    print()

## 2. Zero-Shot Classification

Classify text into custom categories without training data:

In [ ]:
# Zero-shot classification
print("🎯 Zero-Shot Text Classification")
print("=" * 33)

zero_shot_classifier = pipeline("zero-shot-classification")

# Customer service tickets
customer_tickets = [
    "My order hasn't arrived yet and it's been 2 weeks.",
    "The product broke after just one day of use.",
    "I want to return this item for a full refund.",
    "Can you help me track my package?",
    "The app keeps crashing when I try to log in.",
    "Your customer service representative was very helpful!"
]

# Define categories
categories = ["shipping", "product quality", "returns", "technical support", "compliment"]

print(f"Categories: {categories}\n")

for i, ticket in enumerate(customer_tickets, 1):
    result = zero_shot_classifier(ticket, categories)
    
    print(f"{i}. '{ticket}'")
    print(f"   Category: {result['labels'][0]} ({result['scores'][0]:.3f})")
    print(f"   Top 3: {list(zip(result['labels'][:3], [f'{s:.3f}' for s in result['scores'][:3]]))}")
    print()

## 3. Custom Dataset and Fine-tuning

Let's create a custom movie review dataset and fine-tune a model:

In [ ]:
# Create sample movie review dataset
print("Creating Movie Review Dataset")
print("=" * 32)

movie_reviews = {
    'text': [
        "This movie was absolutely fantastic! Great acting and storyline.",
        "Terrible film with poor acting and a confusing plot.",
        "One of the best movies I've ever seen. Highly recommended!",
        "Boring and predictable. Not worth your time.",
        "Amazing cinematography and brilliant performances throughout.",
        "The worst movie I've watched this year. Complete waste of money.",
        "Good movie with some great moments, though a bit slow at times.",
        "Outstanding film that deserves all the awards it received.",
        "Disappointing sequel that doesn't live up to the original.",
        "Incredible story with perfect casting and direction.",
        "Average movie, nothing special but not terrible either.",
        "Absolutely loved it! Will definitely watch it again.",
        "Poor dialogue and weak character development throughout.",
        "Masterpiece of cinema with stunning visuals and sound.",
        "Not impressed. Expected much better from this director.",
        "Excellent acting and a compelling storyline from start to finish.",
        "Terrible editing and pacing made this unwatchable.",
        "Beautiful film that touched my heart. Truly moving experience.",
        "Complete disaster. Couldn't even finish watching it.",
        "Great entertainment value with perfect balance of action and humor."
    ],
    'label': [1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1]  # 1=positive, 0=negative
}

df = pd.DataFrame(movie_reviews)
print(f"Dataset size: {len(df)} samples")
print(f"Label distribution:\n{df['label'].value_counts()}")

# Show sample reviews
print("\nSample reviews:")
for i in range(3):
    label_text = "Positive" if df.iloc[i]['label'] == 1 else "Negative"
    print(f"{i+1}. [{label_text}] '{df.iloc[i]['text']}'")

## 4. Model Evaluation

Let's evaluate classification performance with proper metrics:

In [ ]:
# Evaluate model performance
print("Model Evaluation")
print("=" * 18)

# Use pre-trained model for evaluation demo
classifier = pipeline("sentiment-analysis")

# Get predictions for our dataset
predictions = []
true_labels = []

for text, true_label in zip(df['text'], df['label']):
    pred = classifier(text)[0]
    # Convert model output to binary
    pred_label = 1 if pred['label'] == 'POSITIVE' else 0
    predictions.append(pred_label)
    true_labels.append(true_label)

# Calculate metrics
accuracy = accuracy_score(true_labels, predictions)
print(f"Accuracy: {accuracy:.3f}")

# Classification report
print("\nClassification Report:")
print(classification_report(true_labels, predictions, 
                          target_names=['Negative', 'Positive']))

# Confusion matrix
cm = confusion_matrix(true_labels, predictions)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Negative', 'Positive'],
            yticklabels=['Negative', 'Positive'])
plt.title('Confusion Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.show()

## 5. Multi-label Classification

Handle cases where text can belong to multiple categories:

In [ ]:
# Multi-label classification example
print("Multi-label Classification")
print("=" * 28)

# News articles that can belong to multiple categories
news_articles = [
    "Apple announces new iPhone with advanced AI features for business users.",
    "Climate change affects global economy and technology investments.",
    "New medical AI breakthrough helps doctors diagnose diseases faster.",
    "Sports teams use technology and data analytics to improve performance.",
    "Government announces new policies for technology and healthcare sectors."
]

categories = ["Technology", "Business", "Health", "Politics", "Sports", "Environment"]

multi_label_classifier = pipeline("zero-shot-classification")

print(f"Categories: {categories}\n")

for i, article in enumerate(news_articles, 1):
    result = multi_label_classifier(article, categories, multi_label=True)
    
    print(f"{i}. '{article}'")
    print("    Relevant categories (score > 0.5):")
    
    relevant = [(label, score) for label, score in zip(result['labels'], result['scores']) if score > 0.5]
    
    if relevant:
        for label, score in relevant:
            print(f"       {label}: {score:.3f}")
    else:
        print(f"       Best match: {result['labels'][0]} ({result['scores'][0]:.3f})")
    print()

## 6. Performance Optimization

Compare different models and optimize for speed:

In [ ]:
# Performance comparison
print("Performance Optimization")
print("=" * 26)

import time

# Models to compare
models = [
    ("DistilBERT", "distilbert-base-uncased-finetuned-sst-2-english"),
    ("RoBERTa", "cardiffnlp/twitter-roberta-base-sentiment-latest")
]

test_text = "This movie was absolutely fantastic with great acting!"
batch_texts = [test_text] * 50  # Batch for speed testing

print(f"Test text: '{test_text}'")
print(f"Batch size: {len(batch_texts)} texts\n")

for model_name, model_path in models:
    try:
        print(f"Testing {model_name}:")
        
        # Load model
        start_time = time.time()
        classifier = pipeline("sentiment-analysis", model=model_path)
        load_time = time.time() - start_time
        
        # Single inference
        start_time = time.time()
        result = classifier(test_text)[0]
        single_time = time.time() - start_time
        
        # Batch inference
        start_time = time.time()
        batch_results = classifier(batch_texts)
        batch_time = time.time() - start_time
        
        print(f"   Load time: {load_time:.3f}s")
        print(f"   Single inference: {single_time*1000:.1f}ms")
        print(f"   Batch inference: {batch_time:.3f}s ({batch_time/len(batch_texts)*1000:.1f}ms per text)")
        print(f"   Result: {result['label']} ({result['score']:.3f})")
        print()
        
    except Exception as e:
        print(f"   Error: {str(e)[:50]}...\n")

## 7. Real-world Application

Let's build a comprehensive customer feedback classification system:

In [ ]:
# Customer Feedback Classification System
print("Customer Feedback Classifier")
print("=" * 32)

class FeedbackClassifier:
    def __init__(self):
        self.sentiment_classifier = pipeline("sentiment-analysis")
        self.zero_shot_classifier = pipeline("zero-shot-classification")
        
        self.categories = ["product quality", "shipping", "customer service", 
                          "pricing", "website issues", "returns"]
        self.urgency_levels = ["urgent", "medium", "low"]
    
    def classify(self, text):
        # Sentiment
        sentiment = self.sentiment_classifier(text)[0]
        
        # Category
        category = self.zero_shot_classifier(text, self.categories)['labels'][0]
        
        # Urgency
        urgency = self.zero_shot_classifier(text, self.urgency_levels)['labels'][0]
        
        return {
            'sentiment': sentiment['label'],
            'sentiment_score': sentiment['score'],
            'category': category,
            'urgency': urgency,
            'text': text
        }
    
    def generate_response_plan(self, classification):
        """Generate action plan based on classification"""
        actions = []
        
        if classification['sentiment'] == 'NEGATIVE':
            if classification['sentiment_score'] > 0.8:
                actions.append(" Priority: Immediate attention required")
            actions.append(" Apologize and provide solution")
        else:
            actions.append(" Thank customer for positive feedback")
        
        if classification['urgency'] == 'urgent':
            actions.append(" Respond within 2 hours")
        elif classification['urgency'] == 'medium':
            actions.append(" Respond within 24 hours")
        else:
            actions.append(" Respond within 3 days")
        
        category_actions = {
            'product quality': ' Forward to product team',
            'shipping': ' Contact logistics team',
            'customer service': ' Escalate to supervisor',
            'pricing': ' Review pricing concerns',
            'website issues': ' Forward to tech team',
            'returns': '↩ Process return request'
        }
        
        if classification['category'] in category_actions:
            actions.append(category_actions[classification['category']])
        
        return actions

# Initialize classifier
feedback_classifier = FeedbackClassifier()
print("Feedback classifier initialized!")

In [ ]:
# Test the feedback classifier
test_feedbacks = [
    "The product broke after one day! I'm extremely frustrated and want a refund!",
    "Great product, fast shipping. Very satisfied with my purchase.",
    "The website keeps crashing during checkout. Please fix this issue.",
    "Customer service was unhelpful and rude. Very disappointed.",
    "Good quality but took too long to arrive. Expected faster delivery."
]

print("🔍 Feedback Analysis Results")
print("=" * 28)

for i, feedback in enumerate(test_feedbacks, 1):
    result = feedback_classifier.classify(feedback)
    actions = feedback_classifier.generate_response_plan(result)
    
    print(f"\n Feedback {i}:")
    print(f"'{feedback}'")
    print("-" * 50)
    
    print(f" Classification:")
    print(f"   Sentiment: {result['sentiment']} ({result['sentiment_score']:.3f})")
    print(f"   Category: {result['category']}")
    print(f"   Urgency: {result['urgency']}")
    
    print(f"\n Action Plan:")
    for action in actions:
        print(f"   {action}")

## 🎯 Key Takeaways

**What you've learned about text classification:**

✅ **Pre-trained Models**: Leverage existing models for quick solutions  
✅ **Zero-Shot Classification**: Classify without training data  
✅ **Fine-tuning**: Customize models for specific domains  
✅ **Multi-label Classification**: Handle multiple categories per text  
✅ **Model Evaluation**: Use proper metrics and validation  
✅ **Performance Optimization**: Speed and efficiency considerations  
✅ **Real-world Applications**: Complete classification systems  

## 🔧 Best Practices

1. **Start with pre-trained models** before building custom ones
2. **Use zero-shot classification** for rapid prototyping
3. **Evaluate thoroughly** with proper train/validation splits
4. **Consider batch processing** for production efficiency
5. **Handle edge cases** and class imbalance appropriately
6. **Monitor performance** in production environments

## 🚀 Next Steps

Ready for the next challenge?

**Continue to**: `06_token_classification.ipynb` - Learn about sequence labeling and named entity recognition!

**Practice**: Try building classifiers for your own domain-specific data!

Great work mastering text classification! 🎊